## Machathon 2.0 Qualification Round
### Arabic Poetry Meter

In [44]:
import os
import io
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from random import shuffle
from pyarabic import araby
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional
!pip install pyarabic

### Reading Data

In [45]:
df=pd.read_csv('../input/poemdatasett/train.csv',header=0)
df=pd.read_csv(io.StringIO(u""+df.to_csv(header=None,index=False)), header=None)
df.to_csv("trainFile.txt", header=None, index=None, sep=' ', mode='w')

In [46]:
with open('../input/poemdatasett/labels.txt', 'r') as f:
    data_labels = f.readlines()
    data_labels = [name.replace('\n', '') for name in data_labels]

### Preprocessing Arabic Data

In [47]:
def splitting(path, thresh = 70, on_shatrs = False):
    
    global ALL_WORDS
    words = ""
    X = []
    y = []
    file = open(path, 'r').read()
    file = araby.strip_tashkeel(file)
    removed = '!()*-ـ.:=o[]«»;؛,،~?؟\u200f\ufeffـ'
    after_R = ""
    for char in file:
        if char not in removed:
            after_R += char
    words += after_R
    splitted = after_R.split('\n')
    for line in splitted:
        if len(line) <= 1:
            continue
        data_label, splitted = line.split(' ', 1)
        data_label = int(data_label)
        splitted  = splitted.strip()
        if on_shatrs:
            shatrs = splitted.split('#')
            for shatr in shatrs:
                X.append(shatr.strip())
                y.append(data_label)
        else:
            X.append(splitted.strip())
            y.append(data_label)
    ALL_WORDS = sorted(set(' '.join(X)))  
    X, y = shuffle(X, y)
        
    return X, y

In [48]:
X, y = splitting("./trainFile.txt", on_shatrs=False)

### Showing First Five Abyat

In [49]:
for i in range(5):
    print(X[i], ' ', data_labels[y[i]])

"ولا تجملت بالأرياش مفتخرا # ولا افتتنت بحب الأهل والسكن"   baseet
"لا تأمن الأغفار في النيق أن # تصبح موصولا بها الأحبل"   saree
"ملك في حمى الشبيبة والم # لك له في دنياه زاد الغريب"   khafeef
"وأوظفة أيد جدلها # كأوظفة الفالج المصعب"   mutakareb
"يسرني ويسوء القوم أربعة # أهل وصحب وأعداء وحساد"   baseet


### Splitting to train & validation data

In [50]:
X_train, X_validation , y_train, y_validation = train_test_split(X, y, test_size = 0.18, random_state = 30)

### Converting Arabic Characters to Index

In [51]:
convert_char_idx = {u:i+1 for i, u in enumerate(ALL_WORDS)}

### Making Sequences

In [52]:
def padding(X):
    X = [[convert_char_idx[char] for char in line] for line in X]
    X = pad_sequences(X, padding='post', value=0, maxlen = 100)
    return X

In [53]:
X_train = padding(X_train)
X_validation = padding(X_validation)
y_train = np.array(y_train)
y_validation = np.array(y_validation)

### Building The Model

In [54]:
classifier = Sequential()
classifier.add(Input((100,)))
classifier.add(Embedding(len(convert_char_idx)+1, 256))
classifier.add(Bidirectional(GRU(units = 512, return_sequences=True)))
classifier.add(Bidirectional(GRU(units = 256, return_sequences=True)))
classifier.add(Bidirectional(GRU(units = 256, return_sequences=True)))
classifier.add(Bidirectional(GRU(units = 256)))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(len(data_labels), activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [55]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 256)          10240     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 100, 1024)         2365440   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 100, 512)          1969152   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 100, 512)          1182720   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 512)               1182720   
_________________________________________________________________
dense_4 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)               

In [56]:
classifier(tf.zeros((10, 100))).shape

TensorShape([10, 14])

In [57]:
CB = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
CB += [tf.keras.callbacks.ModelCheckpoint('MAAN_Model', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

### Training The Model

In [58]:
classifier.fit(X_train, y_train, validation_data= (X_validation, y_validation), epochs = 17, batch_size= 64, shuffle = True, callbacks=CB)

Epoch 1/17
257/257 [==============================] - 43s 132ms/step - loss: 2.5002 - accuracy: 0.1068 - val_loss: 2.2889 - val_accuracy: 0.1772

Epoch 00001: val_accuracy improved from -inf to 0.17722, saving model to MAAN_Model
Epoch 2/17
257/257 [==============================] - 32s 125ms/step - loss: 2.2338 - accuracy: 0.1996 - val_loss: 1.9032 - val_accuracy: 0.3681

Epoch 00002: val_accuracy improved from 0.17722 to 0.36806, saving model to MAAN_Model
Epoch 3/17
257/257 [==============================] - 32s 126ms/step - loss: 1.8276 - accuracy: 0.3913 - val_loss: 1.2645 - val_accuracy: 0.5997

Epoch 00003: val_accuracy improved from 0.36806 to 0.59972, saving model to MAAN_Model
Epoch 4/17
257/257 [==============================] - 32s 125ms/step - loss: 1.1430 - accuracy: 0.6449 - val_loss: 0.8848 - val_accuracy: 0.7211

Epoch 00004: val_accuracy improved from 0.59972 to 0.72111, saving model to MAAN_Model
Epoch 5/17
257/257 [==============================] - 32s 126ms/step - 

### Saving The Model

In [59]:
classifier.save('MAAN_Model_Final.h5')

### Loading Model

In [60]:
classifier = tf.keras.models.load_model('MAAN_Model_Final.h5')

### Classifier's Preprocessing Function

In [61]:
def classifier_preprocess_data(text, thresh = 70, on_shatrs = False):
    
    words = ""
    X = []
    text = araby.strip_tashkeel(text)
    removed = 'ـ!()*-ـ.:=o[]«»;؛,،~?؟\u200f\ufeffـ'
    after_R = ""
    for char in text:
        if char not in removed:
            after_R += char
    words += after_R
    splitted = after_R
    splitted  = splitted.strip()
    if on_shatrs:
        shatrs = splitted.split('#')
        for shatr in shatrs:
            X.append(shatr.strip())
    else:
        X.append(splitted.strip())
    X = X[0]
    return X 

In [62]:
data_labels = {i:name for i, name in enumerate(data_labels)}

### Prediction Function

In [63]:
def main_classify_func(sentence):
    sentence = classifier_preprocess_data(sentence, on_shatrs=False)
    sequence = [convert_char_idx[char] for char in sentence]
    sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)
    pred = classifier.predict(sequence)[0]
    print(data_labels[np.argmax(pred, 0).astype('int')], np.max(pred))
    pred = np.argmax(pred, 0).astype('int')
    return pred

### Reading and Classifying Test Data

In [64]:
test_data_df=pd.read_csv('../input/finaltestdata/test (2).csv')
main_classify_func(test_data_df.data[0])

baseet 0.9995615


8

In [70]:
test_data_dictionary = { i: main_classify_func(test_data_df.data[i]) for i in range(test_data_df.shape[0])}
list(test_data_dictionary.keys())[0]

0

In [66]:
test_data_df =  pd.DataFrame(test_data_dictionary.items(), columns=['id', 'labels'])
test_data_df.head()

,id,labels
0,0,8
1,1,2
2,2,2
3,3,10
4,4,11


### Saving Final Predictions to CSV File

In [67]:
test_data_df.to_csv('MAAN_Final.csv', header=True, index=False)